* Saito's post-processing, alpha = 7.2, beta = 0.3
* Adding time_gap feature from Kouki, improves LB by 0.02 (4.327 --> 4.307), CV improves by 0.1
* Incorporate IMU data into the training process, improves LB by 0.014 (4.307 --> 4.293)
* Interpolate the prediction improves LB by 0.096 (4.293 --> 4.197)
* Leakage-based and magnetic Saito's post-processing gave 0.15 improvement (4.197 --> 4.049);
* Use aggregated IMU data, LB: 4.049 --> 4.019;
* Model IMU v4, adding delta waypoints and user ID, LB: 4.019 --> 3.937
* Apply Darich's pp: LB: 3.937 --> ...

# Necessary packages

In [ ]:
import os
from glob import glob
import numpy as np
import math
import pandas as pd
from tqdm.notebook import tqdm
import copy
import pickle as pkl
import json
import matplotlib.pyplot as plt
import multiprocessing

from sklearn.model_selection import GroupKFold
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import scipy
from scipy import optimize
from scipy.optimize import minimize

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW, lr_scheduler

import gc
import warnings
warnings.filterwarnings('ignore')

# Import data

In [ ]:
test_data = pd.read_pickle('../input/indoor-test-set-based-on-wifi/test_all.pkl')   # ../input/indoor-interpolated-with-gap/test_all.pkl
test_data.rename(columns = {'site_id': 'building'}, inplace = True)
test_data['siteid_path'] = ['_'.join([i, j]) for i, j in zip(test_data['building'], test_data['path'])]

In [ ]:
test_data['timestamp'] = [(13 - len(str(i))) * '0' + str(i) for i in test_data['timestamp']]
test_data['site_path_timestamp'] = ['_'.join([i, j, str(k)]) for i, j, k in zip(test_data.building, test_data.path, test_data.timestamp)]

* Add delta waypoints for test data

In [ ]:
test_delta_waypoints = pd.read_csv('../input/indoor-test-set-based-on-wifi/test_all.csv')
test_data = test_data.merge(test_delta_waypoints, left_on = 'site_path_timestamp', right_on = 'site_path_timestamp')
test_data['timestamp'] = test_data['timestamp'].astype(int)

In [ ]:
test_data.head()

In [ ]:
imu_data = pd.read_pickle('../input/ilnaggregated-imu/test_imu_all.pkl')
imu_data['path'] = [i.split('_')[-1] for i in imu_data['site_floor_path']]
imu_data.head()

# Wifi encoder and building map

In [ ]:
with open('../input/iln-wifi-and-building-mapping/label_encoder_bssid.pkl', 'rb') as f:
    lbl_bssid = pkl.load(f)
    
with open('../input/iln-wifi-and-building-mapping/building_map.json', 'r') as f:
    building_map = json.load(f)

bssid_map = dict(zip(lbl_bssid.classes_, lbl_bssid.transform(lbl_bssid.classes_)))

In [ ]:
# Encode BSSID
bssid_features = [i for i in test_data.columns if i.startswith('bssid_')]
rssi_features = [i for i in test_data.columns if i.startswith('rssi_')]
timegap_features = [i for i in test_data.columns if i.startswith('gap_')]
test_data[bssid_features] = test_data[bssid_features].applymap(lambda x: bssid_map[x])

In [ ]:
test_data[bssid_features][test_data[rssi_features] == -999] = 0

In [ ]:
# Encode building
test_data['building'] = test_data.building.map(building_map)
test_data.head()

* Add user ID

In [ ]:
users = pd.read_csv('../input/retrieving-user-id-from-leaked-wifi-feature/df.csv', usecols = ['path_id', 'user_id'])
user_map = dict(zip(users['path_id'], users['user_id']))
test_data['user_id'] = test_data['path'].map(user_map)

# Feature extraction

In [ ]:
def feature_extraction_wifi(x):
    ## Path ID
    path = x['path'].unique()[0]
    
    # Building
    building = x['building'].unique().astype(int)
    
    # User ID
    user = x['user_id'].unique().astype(int)
    
    # BSSID
    bssid_feature = x[bssid_features].values.astype(int)
    
    # RSSI
    rssi_feature = x[rssi_features].values.astype(float)
    for i in range(len(rssi_features)):
        rssi_feature[:,i] = gaussian_filter1d(rssi_feature[:,i], sigma = 2)
    
    # Timegap
    timegap_feature = x[timegap_features].values.astype(float)
    
     # Delta waypoints
    del_waypoints = x[['delta_x_hat', 'delta_y_hat']].values.astype(float)
        
    return path, building, user, bssid_feature, rssi_feature, timegap_feature, del_waypoints

In [ ]:
# Raw IMU
acce_ = ['acce_x', 'acce_y', 'acce_z']
gyro_ = ['gyro_x', 'gyro_y', 'gyro_z']
magn_ = ['magn_x', 'magn_y', 'magn_z']
ahrs_ = ['ahrs_x', 'ahrs_y', 'ahrs_z']
imu_ = acce_ + gyro_ + magn_ + ahrs_

def feature_extraction_imu(df):
    imu = df[imu_].values.astype(int)
    return imu

imu_group = imu_data.groupby('path').apply(feature_extraction_imu)

# Dataset

In [ ]:
class ILN_Dataset(Dataset):
    def __init__(self, group, imu_group, max_len_wifi = 512, max_len_imu = 25_000):
        self.group = group
        self.imu_group = imu_group
        self.max_len_wifi = max_len_wifi
        self.max_len_imu = max_len_imu
        
    def __len__(self):
        return len(self.group)
    
    def __getitem__(self, idx):
        # Extract sequences
        path, building, user, bssid_feature, rssi_feature, timegap_feature, del_waypoints = self.group.iloc[idx]
        
        # Load IMU data
        imu = self.imu_group.loc[path]
        
        seq_len_imu = imu.shape[0]
        mask_imu = np.ones(seq_len_imu)
        mask_imu_ = np.zeros(self.max_len_imu)
        
        # Sequence length
        seq_len_wifi = bssid_feature.shape[0]
        mask = np.ones(seq_len_wifi)
        
        mask_ = np.zeros(self.max_len_wifi)
        bssid_feature_ = np.zeros((self.max_len_wifi, bssid_feature.shape[1]))
        rssi_feature_ = np.zeros((self.max_len_wifi, rssi_feature.shape[1]))
        timegap_feature_ = np.zeros((self.max_len_wifi, timegap_feature.shape[1]))
        del_waypoints_ = np.zeros((self.max_len_wifi, del_waypoints.shape[1]))
        
        imu_ = np.zeros((self.max_len_imu, imu.shape[1]))
        
        if seq_len_wifi <= self.max_len_wifi:   # Pad
            mask_[-seq_len_wifi:] = mask
            bssid_feature_[-seq_len_wifi:,:] = bssid_feature
            rssi_feature_[-seq_len_wifi:,:] = rssi_feature
            timegap_feature_[-seq_len_wifi:,:] = timegap_feature
            del_waypoints_[-seq_len_wifi:,:] = del_waypoints
        else:    # Cut
            mask_ = mask[-self.max_len_wifi:]
            bssid_feature_ = bssid_feature[-self.max_len_wifi:,:]
            rssi_feature_ = rssi_feature[-self.max_len_wifi:,:]
            timegap_feature_ = timegap_feature[-self.max_len_wifi:,:]
            del_waypoints_ = del_waypoints[-self.max_len_wifi:,:]
            
        if seq_len_imu > 0:
            if seq_len_imu <= self.max_len_imu:   # Pad
                mask_imu_[-seq_len_imu:] = mask_imu
                imu_[-seq_len_imu:,:] = imu
            else:    # Cut
                mask_imu_ = mask_imu[-self.max_len_imu:]
                imu_ = imu[-self.max_len_imu:,:]
            
        return {
            'mask': torch.tensor(mask_, dtype = torch.bool),
            'mask_imu': torch.tensor(mask_imu_, dtype = torch.bool),
            'imu': torch.tensor(imu_, dtype = torch.float),
            'building': torch.tensor(building, dtype = torch.long),
            'user': torch.tensor(user, dtype = torch.long),
            'bssid': torch.tensor(bssid_feature_, dtype = torch.long),
            'rssi': torch.tensor(rssi_feature_, dtype = torch.float),
            'timegap': torch.tensor(timegap_feature_, dtype = torch.float),
            'del_waypoints': torch.tensor(del_waypoints_, dtype = torch.float)
        }

# Model

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % nhead == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // nhead
        self.nhead = nhead
        self.linears = clones(nn.Linear(d_model, d_model, bias=False), 4) # Q, K, V, last
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask,
                                 dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
            .view(nbatches, -1, self.nhead * self.d_k)
        return self.linears[-1](x)


class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    
class EncoderLayer(nn.Module):
    """
    Single Encoder block of SAINT
    """
    def __init__(self, d_model, nhead, dim_feedforward = 1024, dropout = 0.1):
        super().__init__()
        self._self_attn = MultiHeadedAttention(d_model, nhead, dropout)
        self._ffn = PositionwiseFeedForward(d_model, dim_feedforward, dropout)
        self._layernorms = clones(nn.LayerNorm(d_model, eps=1e-6), 2)
        self._dropout = nn.Dropout(dropout)

    def forward(self, src, mask = None):
        """
        query: question embeddings
        key: interaction embeddings
        """
        # self-attention block
        src2 = self._self_attn(query=src, key=src, value=src, mask=mask)
        src = src + self._dropout(src2)
        src = self._layernorms[0](src)
        src2 = self._ffn(src)
        src = src + self._dropout(src2)
        src = self._layernorms[1](src)
        return src

* Linformer Layer

In [ ]:
class Lin_MultiHeadedAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1, max_len = 512, target_len = 400):
        "Take in model size and number of heads."
        super(Lin_MultiHeadedAttention, self).__init__()
        assert d_model % nhead == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // nhead
        self.nhead = nhead
        self.linears = clones(nn.Linear(d_model, d_model, bias=False), 4) # Q, K, V, last
        self.linear_key_value = clones(nn.Linear(max_len, 256, bias= False), 3)
        self.linear_query = nn.Linear(max_len, target_len)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(-1)
            # Mask query, key and value
            query = query * mask
            key = key * mask
            mask = value * mask
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.nhead, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Linear projections
        key, value = \
            [l(x).transpose(-1, -2)
             for l, x in zip(self.linear_key_value, (key.transpose(-1,-2), value.transpose(-1,-2)))]
        
        query = self.linear_query(query.transpose(-1, -2)).transpose(-1, -2)

        # 3) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=None,
                                 dropout=self.dropout)

        # 4) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
            .view(nbatches, -1, self.nhead * self.d_k)
        return self.linears[-1](x)
    
class Lin_EncoderLayer(nn.Module):
    """
    Single Encoder block of SAINT
    """
    def __init__(self, d_model, nhead, dim_feedforward = 1024, dropout = 0.1, max_len = 512, target_len = 400):
        super().__init__()
        self._self_attn = Lin_MultiHeadedAttention(d_model, nhead, dropout = dropout, max_len = max_len, target_len = target_len)
        self._ffn = PositionwiseFeedForward(d_model, dim_feedforward, dropout)
        self._layernorms = clones(nn.LayerNorm(d_model, eps=1e-6), 2)
        self._dropout = nn.Dropout(dropout)

    def forward(self, src, mask = None):
        """
        query: question embeddings
        key: interaction embeddings
        """
        # self-attention block
        src2 = self._self_attn(query=src, key=src, value=src, mask=mask)
        src = self._dropout(src2)
        src = self._layernorms[0](src)
        src2 = self._ffn(src)
        src = src + self._dropout(src2)
        src = self._layernorms[1](src)
        return src

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.max_len = max_len
        self.dropout = nn.Dropout(p=dropout)
        self.div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

    def forward(self, x, mask = None):
        if mask is not None:
            position = torch.cumsum(mask.unsqueeze(-1), dim = 1)
        else:
            position = torch.repeat_interleave(torch.arange(0, self.max_len).unsqueeze(-1).unsqueeze(0), x.shape[0], dim = 0)
        
        pe = torch.zeros(x.shape).to(x.device)
        div_term = self.div_term.to(x.device)
        pe[:,:,0::2] = torch.sin(position * div_term)
        pe[:,:,1::2] = torch.cos(position * div_term)
        
        x = x + pe
        
        return self.dropout(x)

In [ ]:
class ILN_Transformer(nn.Module):
    def __init__(self, num_feature_bssid = 100, num_feature_rssi = 100, num_building = 24, num_bssid = 216210, num_user = 27551, 
                 num_floor = 11, num_feature_imu = 12, d_model = 512, nhead = 4, max_len = 512, max_len_imu = 25_000, droprate = 0.1):
        super().__init__()
        self.num_feature_bssid = num_feature_bssid
        self.num_feature_rssi = num_feature_rssi
        self.num_building = num_building
        self.num_bssid = num_bssid
        self.num_user = num_user
        self.num_floor = num_floor
        self.d_model = d_model
        self.nhead = nhead
        self.max_len = max_len
        self.max_len_imu = max_len_imu
        self.droprate = droprate
        
        ############################################ Wifi ############################################
        # Embedding layers
        self.building_embedding = nn.Embedding(num_embeddings = self.num_building, embedding_dim = self.d_model)
        self.user_embedding = nn.Embedding(num_embeddings = self.num_user, embedding_dim = self.d_model)
        self.bssid_embedding = nn.Embedding(num_embeddings = self.num_bssid, embedding_dim = self.d_model // 2, padding_idx = 0)
        
        # Linear layers for BSSID
        self.linear_bssid = nn.Linear(self.num_feature_bssid, 1)
        
        # Linear layers for RSSI and timegap
        self.linear_rssi_timegap = nn.Linear(2, 1)
        self.linear_rssi = nn.Linear(self.num_feature_rssi, self.d_model // 2)
        self.layer_norm_rssi = nn.LayerNorm(self.d_model // 2)
        self.dropout_rssi = nn.Dropout(droprate)
        
        ############################################ IMU ############################################
        # Linear projection
        self.linear_imu_rotation = nn.Sequential(
            nn.Linear(num_feature_imu, d_model // 2),
            nn.LayerNorm(d_model // 2),
            nn.ReLU(),
            nn.Dropout(droprate)
        )
        
        # Positional encoder
        self.position_imu = PositionalEncoding(d_model = self.d_model // 2, dropout = self.droprate, max_len = self.max_len_imu)   # Because the IMU sequences are super long, we need a smaller model size to fit to the memory capacity
        
        # Attention
        self.attn_encoder_imu = Lin_EncoderLayer(d_model = self.d_model // 2, nhead = self.nhead, dim_feedforward = 256, dropout = self.droprate, max_len = self.max_len_imu, target_len = self.max_len)
        
        # self.shrink_time = nn.Linear(self.max_len_imu, self.max_len)
        
        # Output layer
        self.imu_output = nn.LSTM(input_size = d_model // 2, hidden_size = d_model // 2, bidirectional = True, batch_first = True)
        
        ############################################ Concatenate and prediction ############################################
        # Middle linear layers
        self.middle_linear = nn.Sequential(
            nn.Linear(2 * d_model, d_model),
            nn.LayerNorm(d_model),
            nn.ReLU(),
            nn.Dropout(droprate)
        )
        
        # LSTM
        self.lstms = clones(nn.LSTM(input_size = self.d_model, hidden_size = self.d_model // 2, bidirectional = True, batch_first = True), 3)
        
        # Positional encoder
        self.position = PositionalEncoding(d_model = self.d_model, dropout = self.droprate, max_len = self.max_len)
        
        # Attention
        self.attn_encoder = EncoderLayer(d_model = self.d_model, nhead = self.nhead, dim_feedforward = 512, dropout = self.droprate)
        
        self.lstm = nn.LSTM(input_size = self.d_model + 2, hidden_size = self.d_model // 2, bidirectional = True, batch_first = True)
        self.linear_after_encoder = nn.Linear(self.d_model, self.d_model)
        self.layernorm_after_encoder = nn.LayerNorm(self.d_model)
        
        # Output
        self.linear_waypoints = nn.Linear(self.d_model, 2)
        
        # Activation
        self.softmax = nn.Softmax()
        self.prelu = nn.PReLU()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(self.droprate)
        
    def _get_pad_mask(self, seq, pad_idx):
        return (seq == pad_idx).unsqueeze(-2).to(seq.device)
    
    def _get_subsequent_mask(self, seq):
        sz_b, len_s = seq.size()
        subsequent_mask = torch.triu(torch.ones((1, len_s, len_s), device = seq.device), diagonal = 1).bool()
        return subsequent_mask
        
    def forward(self, mask, mask_imu, building, user, bssid, rssi, timegap, imu, del_waypoints):
        ############################################ Wifi ############################################
        # Embed
        building = self.building_embedding(building)
        user = self.user_embedding(user)
        bssid = self.bssid_embedding(bssid)
        
        # BSSID
        bssid = self.linear_bssid(bssid.transpose(-2,-1)).squeeze(-1)
        
        # RSSI and timegap
        rssi = torch.cat((rssi.unsqueeze(-1), timegap.unsqueeze(-1)), dim = -1)
        rssi = self.linear_rssi_timegap(rssi).squeeze(-1)
        rssi = self.linear_rssi(rssi)
        rssi = self.layer_norm_rssi(rssi)
        rssi = self.dropout_rssi(rssi)
        
        ############################################ IMU ############################################
        # Projection
        imu = self.linear_imu_rotation(imu)
        
        # Positional embedding
        imu = self.position_imu(imu, mask = mask_imu)
        
        # Attention
        imu = self.attn_encoder_imu(imu, mask = mask_imu)
        
        # Shrink time
        # imu = self.shrink_time(imu.transpose(-1,-2)).transpose(-1,-2)
        
        # Output IMU
        imu, _ = self.imu_output(imu)
        
        ############################################ Concatenate and prediction ############################################
        x_wifi = torch.cat((bssid, rssi), dim = -1)
        
        x = self.middle_linear(torch.cat((bssid, rssi, imu), dim = -1)) + building + user
        
        for i, layer in enumerate(self.lstms):
            x, _ = layer(x)
        
        # Positional encoding
        x = self.position(x, mask = mask)
        
        # Mask
        mask = ~(self._get_pad_mask(mask, False) | self._get_subsequent_mask(mask))
        
        # Feed it to the Encoder
        x = self.attn_encoder(x, mask)
        
        # Feed it to the lstm
        x, _ = self.lstm(torch.cat((x + x_wifi + imu, del_waypoints), dim = -1))
        
        # Feed over one more linear layer
        x = self.linear_after_encoder(self.dropout(self.prelu(x)))
        x = self.layernorm_after_encoder(x)
        x = self.dropout(self.prelu(x))
        
        # Concatenate floor output and x + building
        x = self.linear_waypoints(x)
        
        return x

# Configuration and util functions

In [ ]:
def infer_fn(model, infer_dataloader, device = 'cpu'):
    model.eval()
    
    pad = []
    way_pred = []
    floor_pred = []
    
    loss = 0
    
    for item in infer_dataloader:
        padding_mask = item['mask'].to(device)
        padding_mask_imu = item['mask_imu'].to(device)
        imu = item['imu'].to(device)
        building = item['building'].to(device)
        user = item['user'].to(device)
        bssid = item['bssid'].to(device)
        rssi = item['rssi'].to(device)
        timegap = item['timegap'].to(device)
        del_waypoints = item['del_waypoints'].to(device)
        
        # Feed input to the model
        with torch.no_grad():
            output_waypoints = model(padding_mask, padding_mask_imu, building, user, bssid, rssi, timegap, imu, del_waypoints)
        
        # Extract non-padded waypoint outputs
        output_waypoints = output_waypoints[padding_mask]
        
        # Store results
        pad.append(padding_mask.cpu().detach().numpy())
        way_pred.append(output_waypoints.cpu().detach().numpy())
        
    # Stack
    pad = np.vstack(pad)
    way_pred = np.vstack(way_pred)
    
    return pad, way_pred

* Configuration

In [ ]:
class config():
    # For inference
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # For dataloader
    batch_size = 64
    num_workers = 4
    # For model
    N = 2    # Number of encoder layers
    d_model = 256
    nhead = 2
    max_len = 400
    max_len_imu = 1024
    droprate = 0.3
    
cfg = config()

# Main

In [ ]:
model_path = '../input/iln-models'

# Dataloader
infer_group = test_data.groupby('siteid_path').apply(feature_extraction_wifi)
infer_dataset = ILN_Dataset(infer_group, imu_group, max_len_wifi = cfg.max_len, max_len_imu = cfg.max_len_imu)
infer_dataloader = DataLoader(infer_dataset, batch_size = cfg.batch_size, num_workers = cfg.num_workers, shuffle = False)

waypoints_prediction = []

for i in tqdm(range(5)):
    ckp = torch.load(os.path.join(model_path, f'model_best_fold_{i}_imu_v4.pt'), map_location = cfg.device)
    model = ILN_Transformer(d_model = cfg.d_model, nhead = cfg.nhead, max_len = cfg.max_len, 
                            max_len_imu = cfg.max_len_imu, droprate = cfg.droprate).to(cfg.device)
    model.load_state_dict(ckp['model_state_dict'])
    padding_mask, waypoints_pred = infer_fn(model, infer_dataloader, device = cfg.device)
    waypoints_prediction.append(waypoints_pred)
    
# Average all folds
waypoints_prediction = np.mean(waypoints_prediction, axis = 0)

# Prediction
prediction = waypoints_prediction

In [ ]:
result = pd.DataFrame(prediction, columns = ['x', 'y'])
result[['site', 'path', 'timestamp']] = test_data[['site', 'path', 'timestamp']].values
result['siteit_path'] = ['_'.join([i, j]) for i, j in zip(result['site'], result['path'])]
result.set_index('siteit_path', inplace = True)

In [ ]:
train_waypoints = pd.read_csv('../input/indoor-location-train-waypoints/train_waypoints.csv')
ss = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
ss[['site','path', 'timestamp']] = [i.split('_') for i in ss.site_path_timestamp]

samples = pd.DataFrame(ss.groupby(['site','path'])['timestamp'].apply(lambda x: list(x)))
buildings = np.unique([x[0] for x in samples.index])
samples.head()

In [ ]:
!git clone --depth 1 https://github.com/location-competition/indoor-location-competition-20 indoor_location_competition_20
!rm -rf indoor_location_competition_20/data

from indoor_location_competition_20.io_f import read_data_file
import indoor_location_competition_20.compute_f as compute_f

In [ ]:
from scipy.interpolate import interp1d
from scipy.ndimage.filters import uniform_filter1d

colacce = ['xyz_time','x_acce','y_acce','z_acce']
colahrs = ['xyz_time','x_ahrs','y_ahrs','z_ahrs']

for building in buildings:
    print(building)
    paths = samples.loc[building].index
    # Acceleration info:
    tfm = pd.read_csv(f'../input/indoor-gbm-postprocessing-xy-prediction/indoor_testing_accel/{building}.txt',index_col = 0)
    for path_id in paths:
        # Original predicted values:
        xy = result.loc[building + '_' + path_id]
        tfmi = tfm.loc[path_id]
        acce_datas = np.array(tfmi[colacce],dtype = np.float)
        ahrs_datas = np.array(tfmi[colahrs],dtype = np.float)
        posi_datas = np.array(xy[['timestamp', 'x', 'y']], dtype = np.float)
        # Outlier removal:
        xyout = uniform_filter1d(posi_datas, size = 3, axis = 0, mode = 'reflect')
        xydiff = np.abs(posi_datas - xyout)
        xystd = np.std(xydiff,axis = 0) * 3
        posi_datas = posi_datas[(xydiff [:,1] < xystd[1]) & (xydiff[:,2] < xystd[2])]
        # Step detection:
        step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
        stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
        # Orientation detection:
        headings = compute_f.compute_headings(ahrs_datas)
        step_headings = compute_f.compute_step_heading(step_timestamps, headings)
        rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
        # Running average:
        posi_datas = uniform_filter1d(posi_datas,size = 3,axis = 0,mode = 'reflect')[0::3,:]
        # The 1st prediction timepoint should be earlier than the 1st step timepoint.
        rel_positions = rel_positions[rel_positions[:,0] > posi_datas[0,0],:]
        # If two consecutive predictions are in-between two step datapoints,
        # the last one is removed, causing error (in the "split_ts_seq" function).
        posi_index = [np.searchsorted(rel_positions[:,0], x, side = 'right') for x in posi_datas[:,0]]
        u, i1, i2 = np.unique(posi_index, return_index = True, return_inverse = True)
        posi_datas = np.vstack([np.mean(posi_datas[i2 == i],axis = 0) for i in np.unique(i2)])
        # Position correction:
        step_positions = compute_f.correct_positions(rel_positions, posi_datas)
        # Interpolate for timestamps in the testing set:
        t = step_positions[:,0]
        x = step_positions[:,1]
        y = step_positions[:,2]
        fx = interp1d(t, x, kind = 'linear', fill_value = (x[0], x[-1]), bounds_error = False) #fill_value="extrapolate"
        fy = interp1d(t, y, kind = 'linear', fill_value = (y[0], y[-1]), bounds_error = False)
        # Output result:
        t0 = np.array(samples.loc[(building,path_id),'timestamp'], dtype = np.float64)
        
        ss.loc[(ss.site == building) & (ss.path == path_id), 'x'] = fx(t0)
        ss.loc[(ss.site == building) & (ss.path == path_id), 'y'] = fy(t0)

ss.head()

* Use floor prediction from Kouki

In [ ]:
floor_prediction = pd.read_csv('../input/indoor-support-data/floor_pred_0507.csv', usecols = ['path', 'floor'])
floor_prediction = dict(zip(floor_prediction['path'], floor_prediction['floor']))
ss['floor'] = ss['path'].map(floor_prediction)
ss['timestamp'] = ss['timestamp'].astype(int)

In [ ]:
ss[['site_path_timestamp', 'floor', 'x', 'y']].to_csv('submission_raw.csv', index = None)

* Visualizing function

In [ ]:
# Helper Functions
def split_col(df):
    df = pd.concat([
        df['site_path_timestamp'].str.split('_', expand=True) \
        .rename(columns={0:'site',
                         1:'path',
                         2:'timestamp'}),
        df
    ], axis=1).copy()
    return df

floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2,
             "F4":3, "F5":4, "F6":5, "F7":6,"F8":7,"F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5,
             "7F":6, "8F": 7, "9F":8}

def plot_preds(
    site,
    floorNo,
    sub=None,
    true_locs=None,
    base="../input/indoor-location-navigation",
    show_train=True,
    show_preds=True,
    show_smoothed=False,
    fix_labels=True,
    map_floor=None
):
    """
    Plots predictions on floorplan map.
    
    map_floor : use a different floor's map
    """
    if map_floor is None:
        map_floor = floorNo
    # Prepare width_meter & height_meter (taken from the .json file)
    floor_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_image.png"
    json_plan_filename = f"{base}/metadata/{site}/{map_floor}/floor_info.json"
    with open(json_plan_filename) as json_file:
        json_data = json.load(json_file)

    width_meter = json_data["map_info"]["width"]
    height_meter = json_data["map_info"]["height"]

    floor_img = plt.imread(f"{base}/metadata/{site}/{map_floor}/floor_image.png")

    fig, ax = plt.subplots(figsize=(12, 12))
    plt.imshow(floor_img)

    if show_train:
        true_locs = true_locs.query('site == @site and floorNo == @map_floor').copy()
        true_locs["x_"] = true_locs["x"] * floor_img.shape[0] / height_meter
        true_locs["y_"] = (
            true_locs["y"] * -1 * floor_img.shape[1] / width_meter
        ) + floor_img.shape[0]
        true_locs.query("site == @site and floorNo == @map_floor").groupby("path").plot(
            x="x_",
            y="y_",
            style="+",
            ax=ax,
            label="train waypoint location",
            color="grey",
            alpha=0.5,
        )

    if show_preds:
        if not show_smoothed:
            sub = sub.query('site == @site and floorNo == @floorNo').copy()
            sub["x_"] = sub["x"] * floor_img.shape[0] / height_meter
            sub["y_"] = (
                sub["y"] * -1 * floor_img.shape[1] / width_meter
            ) + floor_img.shape[0]
            for path, path_data in sub.query(
                "site == @site and floorNo == @floorNo"
            ).groupby("path"):
                path_data.plot(
                    x="x_",
                    y="y_",
                    style=".-",
                    ax=ax,
                    title=f"{site} - floor - {floorNo}",
                    alpha=1,
                    label=path,
                )
        else:
            sub = sub.query('site == @site and floorNo == @floorNo').copy()
            sub["x_"] = sub["x"] * floor_img.shape[0] / height_meter
            sub["y_"] = (
                sub["y"] * -1 * floor_img.shape[1] / width_meter
            ) + floor_img.shape[0]
            for path, path_data in sub.query(
                "site == @site and floorNo == @floorNo"
            ).groupby("path"):
                path_data.plot(
                    x="x_",
                    y="y_",
                    style=".-",
                    ax=ax,
                    title=f"{site} - floor - {floorNo}",
                    alpha=1,
                    label=path,
                )
    if fix_labels:
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(
            by_label.values(), by_label.keys(), loc="center left", bbox_to_anchor=(1, 0.5)
        )
    return fig, ax

def sub_process(sub, train_waypoints):
    train_waypoints['isTrainWaypoint'] = True
    sub = split_col(sub[['site_path_timestamp','floor','x','y']]).copy()
    sub = sub.merge(train_waypoints[['site','floorNo','floor']].drop_duplicates(), how='left')
    sub = sub.merge(
        train_waypoints[['x','y','site','floor','isTrainWaypoint']].drop_duplicates(),
        how='left',
        on=['site','x','y','floor']
             )
    sub['isTrainWaypoint'] = sub['isTrainWaypoint'].fillna(False)
    return sub.copy()

* Visualize the raw prediction

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds=True)
plt.show()

# Saito's post-processing

In [ ]:
ss2 = pd.read_csv('../input/indoorcsv/ILN_594_fold_all_submission_.csv')
ss2[['site', 'path', 'timestamp']] = [i.split('_') for i in ss2.site_path_timestamp]
ss2['timestamp'] = ss2['timestamp'].astype(float)

ss = ss.merge(train_waypoints[['site','floor','floorNo']].drop_duplicates())
ss2 = ss2.merge(train_waypoints[['site','floor','floorNo']].drop_duplicates())

In [ ]:
import math

order = 3
fs = 50.0  # sample rate, Hz
# fs = 100
# cutoff = 3.667  # desired cutoff frequency of the filter, Hz
cutoff = 3

step_distance = 0.8
w_height = 1.7
m_trans = -5

from scipy.signal import butter, lfilter

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def peak_accel_threshold(data, timestamps, threshold):
    d_acc = []
    last_state = 'below'
    crest_troughs = 0
    crossings = []

    for i, datum in enumerate(data):
        
        current_state = last_state
        if datum < threshold:
            current_state = 'below'
        elif datum > threshold:
            current_state = 'above'

        if current_state is not last_state:
            if current_state is 'above':
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)
            else:
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)

            crest_troughs += 1
        last_state = current_state
    return np.array(crossings)

def steps_compute_rel_positions(sample_file):
    
    mix_acce = np.sqrt(sample_file.acce[:,1:2]**2 + sample_file.acce[:,2:3]**2 + sample_file.acce[:,3:4]**2)
    mix_acce = np.concatenate([sample_file.acce[:,0:1], mix_acce], 1)
    mix_df = pd.DataFrame(mix_acce)
    mix_df.columns = ["timestamp","acce"]
    
    filtered = butter_lowpass_filter(mix_df["acce"], cutoff, fs, order)

    threshold = filtered.mean() * 1.1
    crossings = peak_accel_threshold(filtered, mix_df["timestamp"], threshold)

    step_sum = len(crossings)/2
    distance = w_height * 0.4 * step_sum

    mag_df = pd.DataFrame(sample_file.magn)
    mag_df.columns = ["timestamp","x","y","z"]
    
    acce_df = pd.DataFrame(sample_file.acce)
    acce_df.columns = ["timestamp","ax","ay","az"]
    
    mag_df = pd.merge(mag_df,acce_df,on="timestamp")
    mag_df.dropna()
    
    time_di_list = []

    for i in mag_df.iterrows():

        gx,gy,gz = i[1][1],i[1][2],i[1][3]
        ax,ay,az = i[1][4],i[1][5],i[1][6]

        roll = math.atan2(ay,az)
        pitch = math.atan2(-1*ax , (ay * math.sin(roll) + az * math.cos(roll)))

        q = m_trans - math.degrees(math.atan2(
            (gz*math.sin(roll)-gy*math.cos(roll)),(gx*math.cos(pitch) + gy*math.sin(roll)*math.sin(pitch) + gz*math.sin(pitch)*math.cos(roll))
        )) -90
        if q <= 0:
            q += 360
        time_di_list.append((i[1][0],q))

    d_list = [x[1] for x in time_di_list]
    
    steps = []
    step_time = []
    di_dict = dict(time_di_list)

    for n,i in enumerate(crossings[:,:1]):
        if n % 2 == 1:
            continue
        direct_now = di_dict[i[0]]
        dx = math.sin(math.radians(direct_now))
        dy = math.cos(math.radians(direct_now))
#         print(int(n/2+1),"歩目/x:",dx,"/y:",dy,"/角度：",direct_now)
        steps.append((i[0],dx,dy))
        step_time.append(i[0])
    
        step_dtime = np.diff(step_time)/1000
        step_dtime = step_dtime.tolist()
        step_dtime.insert(0,5)
        
        rel_position = []

        wp_idx = 0
#         print("WP:",round(sample_file.waypoint[0,1],3),round(sample_file.waypoint[0,2],3),sample_file.waypoint[0,0])
#         print("------------------")
        for p,i in enumerate(steps):
            step_distance = 0
            if step_dtime[p] >= 1:
                step_distance = w_height*0.25
            elif step_dtime[p] >= 0.75:
                step_distance = w_height*0.3
            elif step_dtime[p] >= 0.5:
                step_distance = w_height*0.4
            elif step_dtime[p] >= 0.35:
                step_distance = w_height*0.45
            elif step_dtime[p] >= 0.2:
                step_distance = w_height*0.5
            else:
                step_distance = w_height*0.4

#             step_x += i[1]*step_distance
#             step_y += i[2]*step_distance
            
            rel_position.append([i[0], i[1]*step_distance, i[2]*step_distance])
#     print(rel_position)
    
    return np.array(rel_position)

In [ ]:
def compute_rel_positions(acce_datas, ahrs_datas):
    step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
    headings = compute_f.compute_headings(ahrs_datas)
    stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
    step_headings = compute_f.compute_step_heading(step_timestamps, headings)
    rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
    return rel_positions

def calc_points(x1_realnum, y1_realnum, x2_realnum, y2_realnum):
    
    x1 = math.floor(x1_realnum)
    y1 = math.floor(y1_realnum)
    x2 = math.floor(x2_realnum)
    y2 = math.floor(y2_realnum)
    
    
    points = []
    
    if (x1 == x2) and (y1 == y2):
        points.append([x1_realnum,y1_realnum])
        points.append([x2_realnum,y2_realnum])
        return np.array(points)

    x1, y1 = x1_realnum, y1_realnum
    x2, y2 = x2_realnum, y2_realnum
    
    step_x = np.sign(x2 - x1)
    step_y = np.sign(y2 - y1)
    a = (y2-y1)/(x2-x1)
    dx = 1/((a**2+1)**0.5)
    dy = a/((a**2+1)**0.5)
    n = math.floor(((y2-y1)**2 + (x2-x1)**2)**0.5)
    
    x = x1_realnum
    y = y1_realnum
    points.append([x1_realnum, y1_realnum])
    for i in range(n-1):
        x = x + step_x * dx
        y = y + step_y * dy
        points.append([x, y])
    points.append([x2_realnum, y2_realnum])
    
    return np.array(points)

def f_coarse(v,xy_hat_,dist): 
    
    length = int(len(xy_hat_)/2)
    
    x = xy_hat_[0::2]
    y = xy_hat_[1::2]
    
    cost = 0.0
    x_ = math.cos(v[2]) * x - math.sin(v[2]) * y + v[0]
    y_ = math.sin(v[2]) * x + math.cos(v[2]) * y + v[1]
    cost_gamma = 0
    for i in range(length-1):
        #points = calc_points(math.floor(x[i]),math.floor(y[i]),math.floor(x[i+1]),math.floor(y[i+1]))
        points = calc_points(x_[i],y_[i],x_[i+1],y_[i+1])
        cost_point = 0
        for point in points:
            gamma = 1000000000
            if point[0] < 0 or point[1] < 0 or point[0] > dist.shape[1] or point[1] > dist.shape[0]:
                cost_point += gamma
            else:
                try:
                    px = point[0]
                    py = point[1]
                    pxf = math.floor(px)
                    pyf = math.floor(py)
                    dist_bilinear = (pxf+1-px)*((pyf+1-py)*dist[pyf][pxf]+(py-pyf)*dist[pyf+1][pxf])+(px-pxf)*((pyf+1-py)*dist[pyf][pxf+1]+(py-pyf)*dist[pyf+1][pxf+1])
                    #cost_gamma += gamma*(dist[point[1]][point[0]])
                    cost_point += gamma*dist_bilinear
                    #print(px,py,dist_bilinear)
                except:
                    cost_point += gamma
        cost_gamma += cost_point/len(points)
    
    cost_alpha = sum((x-x_)**2+(y-y_)**2)
    
    cost = cost_alpha + cost_gamma
    
    return cost

def f_with_(v,path_df, alpha_, beta_, xy_hat_, delta_xy_hat_, dist, delta_t):
    length = int(len(v)/2)-1
    
    dx,dy = v[-2], v[-1]
    x = v[0:-2:2] + dx
    y = v[1:-1:2] + dy
    v = v[0:-2]
    
    #x = v[0::2]
    #y = v[1::2]
    delta_x = delta_xy_hat_[0::2]
    delta_y = delta_xy_hat_[1::2]
    
    cost_alpha = sum(alpha_ * (v - xy_hat_)**2)
    cost_beta = sum(beta_ * (v[2:2*length] - v[0:2*length-2] - delta_xy_hat_)**2)
    
    cost_gamma = 0
    for i in range(length-1):
        points = calc_points(x[i],y[i],x[i+1],y[i+1])
        for j, point in enumerate(points):
            gamma = 100*3
            if j==0 or j==len(points)-1:
                start_end_coeff = 100
            else:
                start_end_coeff = 1
            
            if point[0] < 0 or point[1] < 0 or point[0] > dist.shape[1] or point[1] > dist.shape[0]:
                cost_gamma += gamma * start_end_coeff
            else:
                try:
                    px = point[0]
                    py = point[1]
                    pxf = math.floor(px)
                    pyf = math.floor(py)
                    dist_bilinear = (pxf+1-px)*((pyf+1-py)*dist[pyf][pxf]+(py-pyf)*dist[pyf+1][pxf])+(px-pxf)*((pyf+1-py)*dist[pyf][pxf+1]+(py-pyf)*dist[pyf+1][pxf+1])
                    #cost_gamma += gamma*(dist[point[1]][point[0]])**2
                    cost_gamma += start_end_coeff * gamma*dist_bilinear**2
                    #print(px,py,dist_bilinear)
                except:
                    cost_gamma += gamma * start_end_coeff
        
    return cost_alpha + cost_beta + cost_gamma

In [ ]:
def correct_path(args):
    path, path_df = args
    
    site = path_df['site'].iloc[0]
    floorNo = path_df['floorNo'].iloc[0]
    
    initial_v = np.ravel(path_df[['x', 'y']].values)
    
    T_ref  = path_df['timestamp'].values
    xy_hat = path_df[['x', 'y']].values
    
    example = read_data_file(f'../input/indoor-location-navigation/test/{path}.txt')
    
    rel_positions1 = compute_rel_positions(example.acce, example.ahrs)
    rel_positions2 = steps_compute_rel_positions(example)
    rel1 = rel_positions1.copy()
    rel2 = rel_positions2.copy()
    rel1[:,1:] = rel_positions1[:,1:] / 2
    rel2[:,1:] = rel_positions2[:,1:] / 2
    rel_positions = np.vstack([rel1,rel2])
    rel_positions = rel_positions[np.argsort(rel_positions[:, 0])]
    
    if T_ref[-1] > rel_positions[-1, 0]:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions, np.array([[T_ref[-1], 0, 0]])]
    else:
        rel_positions = [np.array([[0, 0, 0]]), rel_positions]
    rel_positions = np.concatenate(rel_positions)
    
    T_rel = rel_positions[:, 0]
    delta_xy_hat = np.diff(scipy.interpolate.interp1d(T_rel, np.cumsum(rel_positions[:, 1:3], axis=0), axis=0)(T_ref), axis=0)

    N = xy_hat.shape[0]
    delta_t = np.diff(T_ref)
    alpha = (7.2)**(-2) * np.ones(N)
    beta  = (0.3 + 0.3 * 1e-3 * delta_t)**(-2)
    
    alpha_ = np.ravel(np.stack([alpha, alpha],1))
    beta_ = np.ravel(np.stack([beta, beta],1))
    
    xy_hat_ = np.ravel(xy_hat)
    delta_xy_hat_ = np.ravel(delta_xy_hat)
    
    dist = plt.imread(f'../input/indoorlocationnavigationbwdist/metadata/{site}/{floorNo}/geojson_map_cv_dist_x16.png')
    
    x = xy_hat_[0::2]
    y = xy_hat_[1::2]
    
    x2 = ss2[ss2['path'] == path]['x']
    y2 = ss2[ss2['path'] == path]['y']
    initial_v[0::2] = x2
    initial_v[1::2] = y2
    
    initial_v = np.append(initial_v, [0, 0])
    
    v_with_ = minimize(f_with_, x0 = initial_v * 16, args = (path_df, alpha_, beta_, xy_hat_ * 16, delta_xy_hat_ * 16, dist, delta_t), 
                       method = "cobyla", options = {'rhobeg': 1.0, 'maxiter': 10000, 'disp': False, 'catol': 0.0002})
    
    cost = f_with_(v_with_['x'] * 16, path_df, alpha_, beta_, xy_hat_ * 16, delta_xy_hat_ * 16, dist, delta_t)
    
    dx, dy = v_with_['x'][-2], v_with_['x'][-1]
    x_with_ = v_with_['x'][0:-2:2] + dx
    y_with_ = v_with_['x'][1:-1:2] + dy

    return pd.DataFrame({
        'site_path_timestamp' : path_df['site_path_timestamp'],
        'floor' : path_df['floor'],
        'x' : x_with_/16,
        'y' : y_with_/16,
    })

In [ ]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes = processes) as pool:
    dfs = pool.imap_unordered(correct_path, ss.groupby('path'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
ss = pd.concat(dfs).sort_values('site_path_timestamp')

In [ ]:
ss[['site_path_timestamp', 'floor', 'x', 'y']].to_csv('submission_after_Darich.csv', index = None)

* Visualize post-processing after Saito's post processing

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds = True)
plt.show()

# Push to corridor post-processing

In [ ]:
from shapely.geometry import Polygon
from shapely.ops import nearest_points
from shapely.geometry import Point

def fix_prediction(args):
    # Unpack
    (site, floor), df = args
    
    # Find the file path
    floor_name = os.listdir('../input/indoor-location-navigation-scaled-geojson/scaled_geojson/' + site)
    for name in floor_name:
        if floor_map[name] == floor:
            file = '../input/indoor-location-navigation-scaled-geojson/scaled_geojson/' + site + '/' + name + '/shapely_geometry.pkl'
            break
            
    # Open the corridor
    with open(file, 'rb') as f:
        corridor = pkl.load(f)
        
    # Find the outside-corridor points and force them into the corridor
    out_corridor = []
    out_corridor_idx = []
    corridor_nearest_points = []
    for i in range(df.shape[0]):
        p = Point(df[['x', 'y']].iloc[i].values)
        if not p.within(corridor):
            out_corridor.append(p)
            out_corridor_idx.append(df[['x', 'y']].index[i])
            nearest_p, _ = nearest_points(corridor, p)
            x, y = nearest_p.xy[0][0], nearest_p.xy[1][0]
            corridor_nearest_points.append([x, y])
    
    if len(corridor_nearest_points) != 0:
        df.loc[out_corridor_idx, ['x', 'y']] = np.array(corridor_nearest_points)
    
    return df

In [ ]:
ss[['site', 'path', 'timestamp']] = np.array([i.split('_') for i in ss.site_path_timestamp])

processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes = processes) as pool:
    dfs = pool.imap_unordered(fix_prediction, ss.groupby(['site', 'floor']))
    dfs = tqdm(dfs)
    dfs = list(dfs)
ss = pd.concat(dfs).sort_values('site_path_timestamp')[['site_path_timestamp', 'floor', 'x', 'y']]

In [ ]:
ss[['site_path_timestamp', 'floor', 'x', 'y']].to_csv('submission_after_push.csv', index = None)

* Visualize after Push-to-Corridor post-processing

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub, 
           train_waypoints, show_preds = True)
plt.show()

# "Snap to Grid" Post Processing

In [ ]:
def add_xy_site(df):
    df['xy'] = [(x, y) for x,y in zip(df['x'], df['y'])]
    try:
        df['site'] = [i.split('_')[0] for i in df.site_path_timestamp]
    except:
        pass
    return df

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

sub = add_xy_site(ss)
train_waypoints = add_xy_site(train_waypoints)

ds = []
for (site, myfloor), d in tqdm(sub.groupby(['site','floor'])):
    true_floor_locs = train_waypoints.loc[(train_waypoints['floor'] == myfloor) &
                                          (train_waypoints['site'] == site)].reset_index(drop=True)
    if len(true_floor_locs) == 0:
        print(f'Skipping {site} {myfloor}')
        continue
    d['matched_point'] = [closest_point(x, list(true_floor_locs['xy'])) for x in d['xy']]
    d['x_'] = d['matched_point'].apply(lambda x: x[0])
    d['y_'] = d['matched_point'].apply(lambda x: x[1])
    ds.append(d)

sub = pd.concat(ds)

In [ ]:
def snap_to_grid(sub, threshold):
    """
    Snap to grid if within a threshold.
    
    x, y are the predicted points.
    x_, y_ are the closest grid points.
    _x_, _y_ are the new predictions after post processing.
    """
    sub['_x_'] = sub['x']
    sub['_y_'] = sub['y']
    sub.loc[sub['dist'] < threshold, '_x_'] = sub.loc[sub['dist'] < threshold]['x_']
    sub.loc[sub['dist'] < threshold, '_y_'] = sub.loc[sub['dist'] < threshold]['y_']
    return sub.copy()

# Calculate the distances
sub['dist'] = np.sqrt((sub.x-sub.x_)**2 + (sub.y-sub.y_)**2)

ss = snap_to_grid(sub, threshold = 4)

ss = ss[['site_path_timestamp','floor','_x_','_y_']].rename(columns = {'_x_':'x', '_y_':'y'}).sort_values('site_path_timestamp')

In [ ]:
ss[['site_path_timestamp', 'floor', 'x', 'y']].to_csv('submission_after_snap.csv', index = None)

* Visualize after Snap-to-Grip post-processing

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds=True)
plt.show()

# Leakage-based post-processing

In [ ]:
ss['path'] = [i.split('_')[1] for i in ss['site_path_timestamp']]

def device_based_leak_pp(sub):
    df_leak = pd.read_pickle('../input/indoor-support-data/df_leak.pkl')
    df_leak = df_leak.rename({'path_id':'path'}, axis=1)    
    df_sub = sub.copy()
    list_path = df_sub["path"].unique()
    for path in tqdm(list_path):
        df_sub_path = df_sub.query("path == @path")
        start_idx = df_sub.loc[df_sub["path"] == path].index.min()
        end_idx = df_sub.loc[df_sub["path"] == path].index.max()
        start_x = df_sub_path.at[start_idx,"x"]
        start_y = df_sub_path.at[start_idx,"y"]
        end_x = df_sub_path.at[end_idx,"x"]
        end_y = df_sub_path.at[end_idx,"y"]
        start_x_leak = df_leak.query("path == @path")["start_waypoint_x"].iloc[0]
        start_y_leak = df_leak.query("path == @path")["start_waypoint_y"].iloc[0]
        end_x_leak = df_leak.query("path == @path")["end_waypoint_x"].iloc[0]
        end_y_leak = df_leak.query("path == @path")["end_waypoint_y"].iloc[0]
        if not np.isnan(start_x_leak):
            df_sub.at[start_idx,"x"] = start_x_leak
            df_sub.at[start_idx,"y"] = start_y_leak
        if not np.isnan(end_x_leak):
            df_sub.at[end_idx,"x"] = end_x_leak
            df_sub.at[end_idx,"y"] = end_y_leak
    return df_sub

ss = device_based_leak_pp(ss)

In [ ]:
sub = sub_process(ss, train_waypoints)

# Plot the training Data For an example Floor
example_site = '5d2709bb03f801723c32852c'
example_floorNo = 'F4'

plot_preds(example_site, example_floorNo, sub,
           train_waypoints, show_preds=True)
plt.show()

# Submission

In [ ]:
ss[['site_path_timestamp', 'floor', 'x', 'y']].to_csv('submission.csv', index = None)